## Agent Recommender

Uses 2 user functions ai_search and reasoning to simulate multi agent interaction. First user function fetches info from AI search and second user function does reasoning to find patterns. The final agent combines output of both the functions to generate the final response.

In [1]:
from typing import Any

import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentEventHandler,
    FunctionTool,
    MessageDeltaChunk,
    RequiredFunctionToolCall,
    RunStep,
    SubmitToolOutputsAction,
    ThreadMessage,
    ThreadRun,
    ToolOutput,
    CodeInterpreterTool,
)
from azure.identity import DefaultAzureCredential
from my_user_functions import user_functions

from dotenv import load_dotenv
load_dotenv()

https://aistudio8056739269.openai.azure.com/openai/deployments/o1-preview/chat/completions?api-version=2024-08-01-preview
5e7f673e287f429e89b2179944d046bf


True

### Create AI Project Client

In [2]:
# Create a project client
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS


### Create My EventHandler for tools

In [3]:
class MyEventHandler(AgentEventHandler):

    def __init__(self, functions: FunctionTool) -> None:
        super().__init__()
        self.functions = functions
        super().__init__()

    def on_message_delta(self, delta: "MessageDeltaChunk") -> None:
        print(f"Text delta received: {delta.text}")

    def on_thread_message(self, message: "ThreadMessage") -> None:
        print(f"ThreadMessage created. ID: {message.id}, Status: {message.status}")

    def on_thread_run(self, run: "ThreadRun") -> None:
        print(f"ThreadRun status: {run.status}")

        if run.status == "failed":
            print(f"Run failed. Error: {run.last_error}")

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for tool_call in tool_calls:
                print(f"Tool call: {tool_call}")
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        output, my_messages = self.functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                with project_client.agents.submit_tool_outputs_to_stream(
                    thread_id=run.thread_id, run_id=run.id, tool_outputs=tool_outputs, event_handler=self
                ) as stream:
                    stream.until_done()

    def on_run_step(self, step: "RunStep") -> None:
        print(f"RunStep type: {step.type}, Status: {step.status}")

    def on_error(self, data: str) -> None:
        print(f"An error occurred. Data: {data}")

    def on_done(self) -> None:
        print("Stream completed.")

    def on_unhandled_event(self, event_type: str, event_data: Any) -> None:
        print(f"Unhandled Event Type: {event_type}, Data: {event_data}")

### Helper Function to print messages nicely

In [4]:
def print_messages(messages):
    data = messages.get("data")
    data.reverse()
    for d in data:
        print(d.role)
        print(d.content[0].text.value)

In [ ]:
import logging
# Set the logging level for the Azure SDK
logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.WARNING)

### Define the tools that the agent can use

In [5]:
from azure.ai.projects.models import FunctionTool,ToolSet
code_interpreter = CodeInterpreterTool()
functions = FunctionTool(functions=user_functions)
#combined_toolset = code_interpreter.definitions + functions.definitions

#combined_toolset
functions.definitions

toolset = ToolSet()
toolset.add(functions)
toolset.definitions

[{'type': 'function', 'function': {'name': 'insights_from_agent', 'description': "Extract the insights from the user's historical electricity consumption data and also future consumption patterns.", 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'No description'}}, 'required': ['query']}}},
 {'type': 'function', 'function': {'name': 'get_search_results_from_agent', 'description': 'Answer the question related to energy saving suggestions and tips.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The question.'}}, 'required': ['query']}}}]

### Create the agent

In [7]:
prompts = """
You are a helpful assistant designed to make electricity saving tips to customers based on their information and past transaction.
Do not let the customer know the steps you have done.
Here's the steps you must do:
1. You must extract insights from the user's  past and future electricity consumption patterns. You can proceed to step 2 even if you cannot find any insights.
2. You must query the search engine to find energy saving suggestions based on user provided information. 
If the search engine return a list of suggestions, you MUST combine it with the insights extracted from consumption patterns to make the final recommendation.



Always greet the customer.
Do not show any calculations. 
Only use the information from Step 1 and 2 to curate the final response
Do not use emoji.
You leverage code interpreter tool wherever necessary to execute code snippets and provide responses.
"""

In [8]:
agent = project_client.agents.create_agent(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        name="my-assistant",
        description="You take the information from both search agent and insights extractor agent to form final recommendation.",
        instructions=prompts,
        toolset=toolset,
    )

print(f"Created agent, ID: {agent.id}")

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.17.1 Python/3.12.9 (Windows-11-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://eastus.api.azureml.ms/agents/v1.0/subscriptions/92845077-a16a-4db0-b410-b6305ee39257/resourceGroups/dscertsj/providers/Microsoft.MachineLearningServices/workspaces/shilpajain-agent-demo/assistants?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1762'
    'Accept': 'application/json'
    'x-ms-client-request-id': '44f81e6a-e53e-11ef-bd1d-c85ea9ee281f'
    'User-Agent': 'azsdk-python-ai-projects/1.0.0b5 Python/3.1

Created agent, ID: asst_7tEkJOnJrX4HOkGq3dOVD0K0


### Create a  thread

In [9]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID {thread.id}")

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://eastus.api.azureml.ms/agents/v1.0/subscriptions/92845077-a16a-4db0-b410-b6305ee39257/resourceGroups/dscertsj/providers/Microsoft.MachineLearningServices/workspaces/shilpajain-agent-demo/threads?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2'
    'Accept': 'application/json'
    'x-ms-client-request-id': '65ef72d6-e53e-11ef-9b9d-c85ea9ee281f'
    'User-Agent': 'azsdk-python-ai-projects/1.0.0b5 Python/3.12.9 (Windows-11-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 07 Feb 2025 10:29:24 GMT'
    'Content-Type': 'application/json'
    'Content-Length': '137'
    'Connection': 'keep-alive'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'R

Created thread, thread ID thread_FfjzkU14TbUXiFcJciDJThyM


## Create Message

In [10]:
message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, can you help me understand how I consume electricty and also make suggestions on savings.Please note I love cooking ",
    )
print(f"Created message, message ID {message.id}")

with project_client.agents.create_stream(
        thread_id=thread.id, assistant_id=agent.id, event_handler=MyEventHandler(functions)
    ) as stream:
        stream.until_done()

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://eastus.api.azureml.ms/agents/v1.0/subscriptions/92845077-a16a-4db0-b410-b6305ee39257/resourceGroups/dscertsj/providers/Microsoft.MachineLearningServices/workspaces/shilpajain-agent-demo/threads/thread_FfjzkU14TbUXiFcJciDJThyM/messages?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '154'
    'Accept': 'application/json'
    'x-ms-client-request-id': '68299d5f-e53e-11ef-87a3-c85ea9ee281f'
    'User-Agent': 'azsdk-python-ai-projects/1.0.0b5 Python/3.12.9 (Windows-11-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 07 Feb 2025 10:29:27 GMT'
    'Content-Type': 'application/json'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-alive'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'


Created message, message ID msg_hW0vscdkPsHHWhLQcffPGQ3A


INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 07 Feb 2025 10:29:29 GMT'
    'Content-Type': 'text/event-stream; charset=utf-8'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-alive'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'apim-request-id': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'x-ms-region': 'REDACTED'
    'x-aml-cluster': 'REDACTED'
    'x-request-time': 'REDACTED'


ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.IN_PROGRESS
RunStep type: RunStepType.MESSAGE_CREATION, Status: RunStepStatus.IN_PROGRESS
RunStep type: RunStepType.MESSAGE_CREATION, Status: RunStepStatus.IN_PROGRESS
ThreadMessage created. ID: msg_9AgXh4ahXnNnNheVrNkaUkO6, Status: MessageStatus.IN_PROGRESS
ThreadMessage created. ID: msg_9AgXh4ahXnNnNheVrNkaUkO6, Status: MessageStatus.IN_PROGRESS
Text delta received: Sure
Text delta received: ,
Text delta received:  I'd
Text delta received:  be
Text delta received:  happy
Text delta received:  to
Text delta received:  help
Text delta received:  you
Text delta received:  with
Text delta received:  understanding
Text delta received:  your
Text delta received:  electricity
Text delta received:  consumption
Text delta received:  and
Text delta received:  provide
Text delta received:  you
Text delta received:  with
Text delta received:  some
Text delta received:  energy
Text delta received: -s

c:\Users\shilpajain\AppData\Local\Programs\Python\Python312\Lib\site-packages\nixtla\nixtla_client.py:867: UserWarning: Azure endpoint detected, setting `model` to 'azureai'.
  warnings.warn("Azure endpoint detected, setting `model` to 'azureai'.")
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://gptkb-mi223xizmam36.search.windows.net/indexes('vector-2025')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '348'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '801c90bd-e53e-11ef-9832-c85ea9ee281f'
    'User-Agent': 'azsdk-python-search-documents/11.5.2 

...

ThreadRun status: RunStatus.REQUIRES_ACTION
Tool call: {'id': 'call_8Fixbdsmj9eyD5IDyvgPKCxh', 'type': 'function', 'function': {'name': 'insights_from_agent', 'arguments': '{"query": "Extract insights from the user\'s historical and future electricity consumption data"}'}}


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://gptkb-mi223xizmam36.search.windows.net/indexes('vector-2025')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '348'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '9942f15f-e53e-11ef-aa54-c85ea9ee281f'
    'User-Agent': 'azsdk-python-search-documents/11.5.2 Python/3.12.9 (Windows-11-10.0.26100-SP0)'
A body is sent with the request


Error executing tool_call call_8Fixbdsmj9eyD5IDyvgPKCxh: too many values to unpack (expected 2)
Tool call: {'id': 'call_CVs6P4wPfEedOhLhswBCVex0', 'type': 'function', 'function': {'name': 'get_search_results_from_agent', 'arguments': '{"query": "energy saving tips for someone who loves cooking"}'}}


INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Server': 'Microsoft-IIS/10.0'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': '9942f15f-e53e-11ef-aa54-c85ea9ee281f'
    'elapsed-time': 'REDACTED'
    'Date': 'Fri, 07 Feb 2025 10:30:50 GMT'


Output length = 2
• Use a thermal flask to store hot water.

KITCHEN



Cooking Habits
• Cover the pot while cooking. Food cooks faster in covered pots.
• Match the size of the pot with the size of the burner. Energy is lost when small pots are used on larger 

burners.
• Use a microwave oven to cook or warm leftovers instead of a conventional oven.
• Use an over timer instead of opening the oven door to check your food.
• Clean your cooking appliances regularly so that heat can be transferred more efficiently.
• Check the seal on your oven door for cracks or tears to retain heat more effectively.
• Plan your cooking procedures and prepare your ingredients ahead to reduce unnecessary energy usage.

Washing Machine and Dryer
• Wash with an optimal laundry load to maximise energy savings.
• Use the right amount of detergent to avoid washing or rinsing the load unnecessarily.
• Pre-soak or use a soak cycle for heavily soiled garments to avoid two washing cycles.
• Use the economy mode to 

In [11]:
print_messages(project_client.agents.list_messages(thread_id=thread.id))

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://eastus.api.azureml.ms/agents/v1.0/subscriptions/92845077-a16a-4db0-b410-b6305ee39257/resourceGroups/dscertsj/providers/Microsoft.MachineLearningServices/workspaces/shilpajain-agent-demo/threads/thread_FfjzkU14TbUXiFcJciDJThyM/messages?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'c70d0b60-e53e-11ef-8d32-c85ea9ee281f'
    'User-Agent': 'azsdk-python-ai-projects/1.0.0b5 Python/3.12.9 (Windows-11-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 07 Feb 2025 10:32:07 GMT'
    'Content-Type': 'application/json'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-alive'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-reques

MessageRole.USER
Hello, can you help me understand how I consume electricty and also make suggestions on savings.Please note I love cooking 
MessageRole.AGENT
Sure, I'd be happy to help you with understanding your electricity consumption and provide you with some energy-saving suggestions. 

I'll start by analyzing your past and future electricity consumption patterns, and then I'll look up some energy-saving tips particularly for someone who loves cooking.
MessageRole.AGENT
Hello! Based on the analysis of your past and forecasted electricity consumption patterns, as well as some specific energy-saving tips for someone who loves cooking, I have some insights and suggestions for you.

### Insights from Your Electricity Consumption Patterns:

**2024 Historical Patterns:**
1. **Anomalously Low Consumption in February:** February 2024 shows significantly lower consumption compared to other months.
2. **Increasing Trend from March to June:** Consumption rises steadily during these months, l